In [16]:
!pip install ipywidgets

You should consider upgrading via the '/home/sam/Documents/iot-irrigation-kajanja/iot-irrigation-system/lab-experiment/python-analysis/venv/bin/python3 -m pip install --upgrade pip' command.


In [17]:
############################################
# Import required libraries for Irrigation System
############################################

# Data handling and numerical operations
import numpy as np
from datetime import datetime, timedelta
from dataclasses import dataclass
import json

# Type hints and abstract classes
from typing import Dict, Tuple, Optional, List
from abc import ABC, abstractmethod

# System operations and timing
import time

# Database management
import sqlite3

# Logging configuration
import logging

# Jupyter notebook interface components
import ipywidgets as widgets
from IPython.display import display, clear_output

# Configure logging system with both file and console output
logging.basicConfig(
    level=logging.INFO,  # Set logging level
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',  # Define log format
    handlers=[
        logging.FileHandler('irrigation_system.log'),  # Save logs to file
        logging.StreamHandler()  # Display logs in console
    ]
)
# Create logger instance for the Irrigation System
logger = logging.getLogger('IrrigationSystem')

In [18]:
@dataclass
class SensorSpecs:
    """
    Sensor specifications and operational parameters for irrigation sensors.

    Attributes
    ----------
    range_min : float
        Minimum valid sensor reading value (e.g., 0 for moisture, -55 for temp)
    range_max : float
        Maximum valid sensor reading value (e.g., 100 for moisture, 125 for temp)
    accuracy : float
        Sensor measurement accuracy (± units) for uncertainty calculations
    sampling_rate : float
        Time interval between sensor readings in seconds. Lower values = more frequent readings

    Notes
    -----
    Used for sensor validation and uncertainty handling in the irrigation system.
    """
    range_min: float
    range_max: float
    accuracy: float
    sampling_rate: float  # in seconds

In [19]:
@dataclass
class SensorData:
    """
    Container class for storing sensor readings with corresponding timestamp.

    Attributes
    ----------
    soil_moisture : float
        Soil moisture level in percentage (0-100%)
    temperature : float
        Ambient temperature in degrees Celsius
    humidity : float
        Relative humidity in percentage (0-100%)
    water_level : Optional[float]
        Water level in millimeters, may be None if sensor not present
    flow_rate : float
        Water flow rate in liters per minute
    timestamp : datetime
        Timestamp when sensor readings were taken

    Notes
    -----
    Future planned improvements:
    * Solar radiation sensor (W/m²) for more accurate ETo calculation
    * Wind speed sensor (m/s) for improved evaporation estimates
    * Rainfall sensor for precipitation tracking
    * Soil temperature sensor for root zone monitoring

    Example
    -------
    >>> sensor_data = SensorData(
    ...     soil_moisture=65.5,
    ...     temperature=23.4,
    ...     humidity=75.0,
    ...     water_level=30.0,
    ...     flow_rate=15.5,
    ...     timestamp=datetime.now()
    ... )
    """
    soil_moisture: float
    temperature: float
    humidity: float
    water_level: Optional[float]
    flow_rate: float
    timestamp: datetime

In [20]:
class SensorConfig:
    """
    Configuration class defining specifications for all sensors in the irrigation system.

    Attributes
    ----------
    SPECS : dict
        Dictionary mapping sensor types to their specifications with format:
        {sensor_name: SensorSpecs(range_min, range_max, accuracy, sampling_rate)}

        Current Sensors
        --------------
        soil_moisture : SensorSpecs
            Range: 0-100%, Accuracy: ±2.0%, Sampling: 10s
        temperature : SensorSpecs
            Range: -55-125°C, Accuracy: ±0.5°C, Sampling: 10s
        humidity : SensorSpecs
            Range: 0-100%, Accuracy: ±4.5%, Sampling: 10s
        water_level : SensorSpecs
            Range: 0-100mm, Accuracy: ±1.0mm, Sampling: 1s
        flow_rate : SensorSpecs
            Range: 1-30L/min, Accuracy: ±0.02L/min, Sampling: 1s

    Notes
    -----
    Planned future sensor additions:
    * solar_radiation: 0-1500 W/m², Accuracy: ±10 W/m², Sampling: 10s
    * wind_speed: 0-50 m/s, Accuracy: ±0.5 m/s, Sampling: 10s
    * rainfall: 0-500mm, Accuracy: ±0.2mm, Sampling: 1s
    * soil_temp: -10-50°C, Accuracy: ±0.5°C, Sampling: 10s

    Example
    -------
    >>> soil_moisture_specs = SensorConfig.SPECS["soil_moisture"]
    >>> print(f"Soil moisture accuracy: ±{soil_moisture_specs.accuracy}%")
    Soil moisture accuracy: ±2.0%
    """
    SPECS = {
        "soil_moisture": SensorSpecs(0, 100, 2.0, 10),
        "temperature": SensorSpecs(-55, 125, 0.5, 10),
        "humidity": SensorSpecs(0, 100, 4.5, 10),
        "water_level": SensorSpecs(0, 100, 1.0, 1),
        "flow_rate": SensorSpecs(1, 30, 0.02, 1),
    }

In [21]:
class DatabaseManager:
    """
    Database management class for storing and retrieving irrigation system data.

    This class handles all database operations including initialization,
    sensor data logging, and irrigation event tracking.

    Attributes
    ----------
    db_path : str
        Path to the SQLite database file (default: 'irrigation_data.db')

    Notes
    -----
    Currently maintains two tables:
    * sensor_data: Stores periodic sensor readings
    * irrigation_events: Stores irrigation operation records

    Planned improvements:
    * Add sensor calibration table
    * Add system configuration table
    * Implement data archiving
    * Add error log table
    """

    def __init__(self, db_path: str = "irrigation_data.db"):
        """
        Initialize DatabaseManager with specified database path.

        Parameters
        ----------
        db_path : str, optional
            Path to SQLite database file (default: 'irrigation_data.db')
        """
        self.db_path = db_path
        self._init_database()

    def _init_database(self):
        """
        Initialize database tables if they don't exist.

        Creates two tables:
        - sensor_data: For storing sensor readings
            * timestamp (DATETIME)
            * soil_moisture (REAL)
            * temperature (REAL)
            * humidity (REAL)
            * water_level (REAL)
            * flow_rate (REAL)

        - irrigation_events: For storing irrigation operations
            * timestamp (DATETIME)
            * crop_type (TEXT)
            * duration (REAL)
            * water_volume (REAL)
            * reason (TEXT)
        """
        with sqlite3.connect(self.db_path) as conn:
            conn.execute("""
                CREATE TABLE IF NOT EXISTS sensor_data (
                    timestamp DATETIME,
                    soil_moisture REAL,
                    temperature REAL,
                    humidity REAL,
                    water_level REAL,
                    flow_rate REAL
                )
            """)
            conn.execute("""
                CREATE TABLE IF NOT EXISTS irrigation_events (
                    timestamp DATETIME,
                    crop_type TEXT,
                    duration REAL,
                    water_volume REAL,
                    reason TEXT
                )
            """)
            conn.commit()

    def log_sensor_data(self, data: SensorData):
        """
        Log sensor readings to database.

        Parameters
        ----------
        data : SensorData
            Container object with sensor readings and timestamp
        """
        with sqlite3.connect(self.db_path) as conn:
            conn.execute("""
                INSERT INTO sensor_data VALUES (?, ?, ?, ?, ?, ?)
            """, (
                data.timestamp, data.soil_moisture, data.temperature,
                data.humidity, data.water_level, data.flow_rate
            ))
            conn.commit()

    def log_irrigation_event(self, crop_type: str, duration: float,
                           water_volume: float, reason: str):
        """
        Log irrigation event to database.

        Parameters
        ----------
        crop_type : str
            Type of crop being irrigated
        duration : float
            Duration of irrigation in seconds
        water_volume : float
            Volume of water used in liters
        reason : str
            Reason for irrigation event
        """
        with sqlite3.connect(self.db_path) as conn:
            conn.execute("""
                INSERT INTO irrigation_events VALUES (?, ?, ?, ?, ?)
            """, (datetime.now(), crop_type, duration, water_volume, reason))
            conn.commit()

In [22]:
class SimplifiedETo:
    """
    Simplified calculation of reference evapotranspiration (ETo) with Nyeri-Kenya adaptations.

    This class implements a simplified Hargreaves-Samani equation for ETo calculation,
    with optional adjustments for Nyeri, Kenya's specific climatic conditions.

    Notes
    -----
    Nyeri Characteristics:
    * Altitude: ~1,750 meters above sea level
    * Location: Near equator (0°25'S)
    * Climate: Highland climate with two rainy seasons
    * Temperature range: 10-26°C typical
    * Relative Humidity: 60-84% typical

    Examples
    --------
    >>> calculator = SimplifiedETo()
    >>> eto_value, method = calculator.calc_eto(temp=20, humidity=70, elevation=1750, location_adjust=True)
    >>> seasonal_factor = calculator.get_nyeri_seasonal_factor(month=1)
    >>> final_eto = eto_value * seasonal_factor
    """
    
    @staticmethod
    def calc_eto(temp: float, humidity: float, elevation: float = 100, 
                 fixed_eto: float = None, location_adjust: bool = False) -> Tuple[float, str]:
        """
        Calculate reference evapotranspiration (ETo) using simplified Hargreaves-Samani.

        Parameters
        ----------
        temp : float
            Mean temperature in Celsius
        humidity : float
            Relative humidity in percentage (0-100)
        elevation : float, optional
            Site elevation in meters above sea level, defaults to 100
        fixed_eto : float, optional
            Override calculated ETo with fixed value, defaults to None
        location_adjust : bool, optional
            Apply Nyeri-specific adjustments, defaults to False

        Returns
        -------
        Tuple[float, str]
            - ETo value in mm/day
            - Method used ('fixed' or 'calculated')

        Notes
        -----
        Nyeri-specific adjustments (when location_adjust=True):
        * Minimum temperature range: 8.0°C
        * Highland-specific elevation factor
        * Humidity coefficient: 0.12
        * Output range: 2.5-12.0 mm/day
        """
        if fixed_eto is not None:
            return fixed_eto, "fixed"
            
        # Constants
        SOLAR_CONSTANT = 0.082  # MJ/m²/min
        
        # Estimate temperature range based on humidity
        min_temp_range = 8.0 if location_adjust else 5.0
        temp_range = max(min_temp_range, 12.0 * (1 - humidity/100))
        
        # Extra-terrestrial radiation (Ra) approximation
        if location_adjust:
            elevation_factor = 1 + (elevation/8000)  # Highland adjustment
        else:
            elevation_factor = 1 + elevation/10000   # Standard adjustment
            
        ra = SOLAR_CONSTANT * elevation_factor * 24 * 60 * 0.4  # MJ/m²/day
        ra_mm = ra * 0.408  # Convert to mm/day
        
        # Hargreaves-Samani equation
        eto = 0.0023 * (temp + 17.8) * (temp_range ** 0.5) * ra_mm
        
        # Humidity correction
        humidity_coef = 0.12 if location_adjust else 0.15
        humidity_factor = 1.0 - humidity_coef * (humidity/100)
        
        min_factor = 0.88 if location_adjust else 0.85
        eto *= max(min_factor, min(humidity_factor, 1.0))
        
        # Output constraints
        if location_adjust:
            return max(2.5, min(eto, 12.0)), "calculated"
        else:
            return max(2.0, min(eto, 15.0)), "calculated"

    @staticmethod
    def get_nyeri_seasonal_factor(month: int) -> float:
        """
        Get seasonal adjustment factor for Nyeri's climate patterns.

        Parameters
        ----------
        month : int
            Month of the year (1-12)

        Returns
        -------
        float
            Seasonal adjustment factor based on Nyeri's seasons:
            * Hot dry season (Jan-Feb): 1.1
            * Long rains (Mar-May): 0.9
            * Cool dry season (Jun-Sep): 1.0
            * Short rains (Oct-Dec): 0.9

        Examples
        --------
        >>> factor = SimplifiedETo.get_nyeri_seasonal_factor(1)  # January
        >>> print(factor)  # Returns 1.1
        """
        seasonal_factors = {
            1: 1.1, 2: 1.1,  # Hot dry
            3: 0.9, 4: 0.9, 5: 0.9,  # Long rains
            6: 1.0, 7: 1.0, 8: 1.0, 9: 1.0,  # Cool dry
            10: 0.9, 11: 0.9, 12: 0.9  # Short rains
        }
        return seasonal_factors.get(month, 1.0)

    @staticmethod
    def is_valid_nyeri_conditions(temp: float, humidity: float, 
                                elevation: float) -> bool:
        """
        Validate if weather conditions are within typical Nyeri ranges.

        Parameters
        ----------
        temp : float
            Temperature in Celsius
        humidity : float
            Relative humidity in percentage
        elevation : float
            Site elevation in meters

        Returns
        -------
        bool
            True if all conditions are within Nyeri's typical ranges:
            * Temperature: 10-26°C
            * Humidity: 60-84%
            * Elevation: 1400-2500m

        Examples
        --------
        >>> is_valid = SimplifiedETo.is_valid_nyeri_conditions(
        ...     temp=20, humidity=70, elevation=1750
        ... )
        >>> print(is_valid)  # Returns True
        """
        return (10 <= temp <= 26 and 
                60 <= humidity <= 84 and 
                1400 <= elevation <= 2500)

In [23]:
class ValveController:
    """
    Controls irrigation valves with safety checks and timing management.

    This class manages the operation of irrigation valves, including opening,
    closing, and safety timeout features to prevent over-irrigation.

    Attributes
    ----------
    valve_states : dict
        Dictionary tracking open valves and their opening timestamps
    max_open_time : int
        Maximum allowed valve open duration in seconds (default: 3600/1 hour)

    Notes
    -----
    Future hardware implementation should add actual valve control code
    in open_valve and close_valve methods.
    """

    def __init__(self):
        """
        Initialize the valve controller with empty states and default timeout.
        
        The controller starts with no open valves and sets a maximum valve
        open time of 1 hour (3600 seconds) for safety.
        """
        self.valve_states = {}
        self.max_open_time = 3600  # Maximum time a valve can stay open (1 hour)

    def open_valve(self, valve_id: str) -> bool:
        """
        Open specified irrigation valve with safety timeout check.

        Parameters
        ----------
        valve_id : str
            Unique identifier for the valve to open

        Returns
        -------
        bool
            True if valve was successfully opened
            False if valve was forced to close due to timeout

        Notes
        -----
        Performs a safety check to ensure the valve hasn't been open
        beyond the maximum allowed time (3600 seconds) before opening.
        """
        if valve_id in self.valve_states:
            if datetime.now() - self.valve_states[valve_id] > timedelta(seconds=self.max_open_time):
                logger.warning(f"Valve {valve_id} has been open too long - forcing close")
                self.close_valve(valve_id)
                return False

        # In real implementation, add hardware control code here
        self.valve_states[valve_id] = datetime.now()
        logger.info(f"Opened valve {valve_id}")
        return True

    def close_valve(self, valve_id: str) -> None:
        """
        Close specified irrigation valve.

        Parameters
        ----------
        valve_id : str
            Unique identifier for the valve to close

        Notes
        -----
        Removes the valve from active states tracking after closing.
        Future implementation should add actual hardware control code.
        """
        # In real implementation, add hardware control code here
        if valve_id in self.valve_states:
            del self.valve_states[valve_id]
        logger.info(f"Closed valve {valve_id}")

In [24]:
class CropManager:
    """
    Manages crop-specific parameters and growth stage calculations.

    This class handles crop parameters, growth stages, and their coefficients
    for different types of crops in the irrigation system.

    Attributes
    ----------
    crop_type : str
        Type of crop being managed (beans, maize, onions, or rice)
    planting_date : datetime
        Date when the crop was planted
    parameters : dict
        Dictionary of crop-specific parameters including moisture and temperature limits
    stages : dict
        Dictionary of growth stages with their coefficients and durations

    Notes
    -----
    Supports four crop types: beans, maize, onions, and rice.
    Each crop has specific moisture, temperature, and humidity requirements.
    Growth stages include: initial, development, mid_season, and late_season.
    """

    def __init__(self, crop_type: str, planting_date: datetime):
        """
        Initialize crop manager with specific crop type and planting date.

        Parameters
        ----------
        crop_type : str
            Type of crop to manage (beans, maize, onions, or rice)
        planting_date : datetime
            Date when the crop was planted
        """
        self.crop_type = crop_type
        self.planting_date = planting_date
        
        # Crop parameters
        self.parameters = {
            "beans": {
                "moisture_min": 65,    # Minimum soil moisture (%)
                "moisture_max": 85,    # Maximum soil moisture (%)
                "temp_max": 26,        # Maximum temperature (°C)
                "humidity_target": 85,  # Target humidity (%)
                "water_level": None    # Water level range (mm)
            },
            "maize": {
                "moisture_min": 55,
                "moisture_max": 65,
                "temp_max": 30,
                "humidity_target": 55,
                "water_level": None
            },
            "onions": {
                "moisture_min": 50,
                "moisture_max": 65,
                "temp_max": 24,
                "humidity_target": 70,
                "water_level": None
            },
            "rice": {
                "moisture_min": 95,
                "moisture_max": 100,
                "temp_max": 37,
                "humidity_target": 60,
                "water_level": (15, 45)  # (min, max) water level in mm
            }
        }
        
        # Growth stages and crop coefficients
        self.stages = {
            "beans": {
                "initial": (0.35, 15),      # (coefficient, duration in days)
                "development": (0.7, 25),
                "mid_season": (1.1, 35),
                "late_season": (0.3, 20)
            },
            "maize": {
                "initial": (0.4, 20),
                "development": (0.8, 35),
                "mid_season": (1.15, 40),
                "late_season": (0.7, 30)
            },
            "onions": {
                "initial": (0.5, 15),
                "development": (0.8, 25),
                "mid_season": (1.05, 70),
                "late_season": (0.85, 40)
            },
            "rice": {
                "initial": (1.1, 60),
                "development": (0, 0),
                "mid_season": (1.2, 60),
                "late_season": (1.0, 30)
            }
        }

    def get_current_stage(self) -> Tuple[str, float]:
        """
        Determine current growth stage and crop coefficient based on planting date.

        Returns
        -------
        tuple[str, float]
            - Current growth stage name (initial, development, mid_season, or late_season)
            - Corresponding crop coefficient (kc) for the stage

        Notes
        -----
        Calculates days since planting and determines current stage based on
        cumulative duration of stages. If beyond all stages, returns late_season.

        Examples
        --------
        >>> manager = CropManager("beans", datetime(2024, 1, 1))
        >>> stage, coefficient = manager.get_current_stage()
        >>> print(f"Current stage: {stage}, Coefficient: {coefficient}")
        """
        days_since_planting = (datetime.now() - self.planting_date).days
        cumulative_days = 0
        
        for stage, (kc, duration) in self.stages[self.crop_type].items():
            cumulative_days += duration
            if days_since_planting <= cumulative_days:
                return stage, kc
        
        # If beyond all stages, return last stage
        return "late_season", self.stages[self.crop_type]["late_season"][0]

In [25]:
class IrrigationController:
    """
    Main irrigation control system for managing automated irrigation operations.

    This class integrates sensor readings, crop requirements, valve control,
    and safety monitoring for automated irrigation management.

    Attributes
    ----------
    crop_manager : CropManager
        Manages crop-specific parameters and growth stages
    valve_controller : ValveController
        Controls irrigation valve operations
    db_manager : DatabaseManager
        Handles data logging and storage
    plot_area : float
        Area of irrigation plot in square meters
    valve_id : str
        Identifier for the associated irrigation valve
    measurement_window : list
        Buffer for sensor measurements for uncertainty handling
    window_size : int
        Number of measurements to maintain in the window (default: 5)
    """

    def __init__(self, crop_type: str, plot_dimensions: Tuple[float, float],
                 planting_date: datetime, valve_id: str):
        """
        Initialize irrigation controller with plot and crop specifications.

        Parameters
        ----------
        crop_type : str
            Type of crop being irrigated
        plot_dimensions : tuple[float, float]
            Plot dimensions (width, length) in meters
        planting_date : datetime
            Date when crop was planted
        valve_id : str
            Identifier for irrigation valve
        """
        self.crop_manager = CropManager(crop_type, planting_date)
        self.valve_controller = ValveController()
        self.db_manager = DatabaseManager()
        self.plot_area = plot_dimensions[0] * plot_dimensions[1]
        self.valve_id = valve_id
        
        # Initialize uncertainty handling
        self.measurement_window = []
        self.window_size = 5  # Number of measurements to consider for uncertainty

    def read_sensors(self) -> SensorData:
        """
        Read all sensors with simulated uncertainty.

        Returns
        -------
        SensorData
            Container with current sensor readings

        Notes
        -----
        Currently implements simulated readings. Future improvements:
        * Add integration with actual hardware sensors
        * Implement sensor calibration routines
        * Add sensor health monitoring
        * Include data quality checks
        * Add support for wireless sensors
        * Implement sensor redundancy
        """
        base_moisture = np.random.normal(
            (self.crop_manager.parameters[self.crop_manager.crop_type]["moisture_min"] +
            self.crop_manager.parameters[self.crop_manager.crop_type]["moisture_max"]) / 2,
            SensorConfig.SPECS["soil_moisture"].accuracy
        )
        
        data = SensorData(
            soil_moisture=base_moisture,
            temperature=np.random.normal(25, SensorConfig.SPECS["temperature"].accuracy),
            humidity=np.random.normal(60, SensorConfig.SPECS["humidity"].accuracy),
            water_level=np.random.normal(30, SensorConfig.SPECS["water_level"].accuracy),
            flow_rate=np.random.normal(15, SensorConfig.SPECS["flow_rate"].accuracy),
            timestamp=datetime.now()
        )
        
        return data

    def handle_measurement_uncertainty(self, data: SensorData) -> SensorData:
        """
        Apply moving average filter for measurement uncertainty reduction.

        Parameters
        ----------
        data : SensorData
            Raw sensor readings

        Returns
        -------
        SensorData
            Filtered sensor readings

        Notes
        -----
        Future improvements:
        * Add outlier detection
        * Implement Kalman filtering
        * Add sensor drift compensation
        * Include cross-sensor validation
        """
        self.measurement_window.append(data)
        if len(self.measurement_window) > self.window_size:
            self.measurement_window.pop(0)
        
        avg_data = SensorData(
            soil_moisture=np.mean([d.soil_moisture for d in self.measurement_window]),
            temperature=np.mean([d.temperature for d in self.measurement_window]),
            humidity=np.mean([d.humidity for d in self.measurement_window]),
            water_level=np.mean([d.water_level for d in self.measurement_window]) if data.water_level is not None else None,
            flow_rate=np.mean([d.flow_rate for d in self.measurement_window]),
            timestamp=data.timestamp
        )
        return avg_data

    def calculate_irrigation_need(self, sensor_data: SensorData) -> Tuple[bool, float, float]:
        """
        Calculate irrigation requirements based on current conditions.

        Parameters
        ----------
        sensor_data : SensorData
            Current sensor readings

        Returns
        -------
        tuple[bool, float, float]
            - Whether irrigation is needed
            - Required irrigation duration (seconds)
            - Current moisture deficit (%)

        Notes
        -----
        Future improvements:
        * Add weather forecast integration
        * Implement soil type adjustments
        * Add crop stress indicators
        * Include historical irrigation patterns
        * Add machine learning for optimization
        """
        stage, kc = self.crop_manager.get_current_stage()
        eto, _ = SimplifiedETo.calc_eto(sensor_data.temperature, sensor_data.humidity)
        daily_requirement = float(eto) * float(kc) * float(self.plot_area)
        
        params = self.crop_manager.parameters[self.crop_manager.crop_type]
        target_moisture = (params["moisture_min"] + params["moisture_max"]) / 2
        moisture_deficit = target_moisture - sensor_data.soil_moisture
        
        if moisture_deficit > 0:
            duration = (float(daily_requirement) * float(moisture_deficit) / 100.0) / float(sensor_data.flow_rate)
            return True, min(float(duration), 3600.0), moisture_deficit
        
        return False, 0.0, moisture_deficit

    def control_loop(self):
        """
        Main control loop for irrigation system.

        Continuously monitors conditions and controls irrigation based on:
        * Sensor readings
        * Crop requirements
        * Safety checks
        * Abnormal conditions

        Notes
        -----
        Includes error handling and emergency valve closure.
        """
        while True:
            try:
                raw_data = self.read_sensors()
                sensor_data = self.handle_measurement_uncertainty(raw_data)
                need_irrigation, duration, moisture_deficit = self.calculate_irrigation_need(sensor_data)
                
                if need_irrigation and self._safety_checks(sensor_data):
                    logger.info(f"Starting irrigation for {duration:.1f} seconds")
                    
                    if self.valve_controller.open_valve(self.valve_id):
                        irrigation_start_time = datetime.now()
                        try:
                            self._monitor_irrigation(duration, sensor_data)
                        finally:
                            self.valve_controller.close_valve(self.valve_id)
                        
                        actual_duration = (datetime.now() - irrigation_start_time).total_seconds()
                        water_volume = actual_duration * sensor_data.flow_rate / 60
                        
                        self.db_manager.log_irrigation_event(
                            self.crop_manager.crop_type,
                            actual_duration,
                            water_volume,
                            f"Moisture deficit: {moisture_deficit:.1f}%"
                        )
                
                time.sleep(SensorConfig.SPECS["soil_moisture"].sampling_rate)
                
            except Exception as e:
                logger.error(f"Error in control loop: {str(e)}")
                self.valve_controller.close_valve(self.valve_id)
                time.sleep(60)

    def _safety_checks(self, sensor_data: SensorData) -> bool:
        """
        Perform safety checks before irrigation.

        Parameters
        ----------
        sensor_data : SensorData
            Current sensor readings

        Returns
        -------
        bool
            True if all safety checks pass, False otherwise
        """
        try:
            for param, value in {
                "soil_moisture": sensor_data.soil_moisture,
                "temperature": sensor_data.temperature,
                "humidity": sensor_data.humidity,
                "flow_rate": sensor_data.flow_rate
            }.items():
                specs = SensorConfig.SPECS[param]
                if not specs.range_min <= value <= specs.range_max:
                    logger.warning(f"Invalid {param} reading: {value}")
                    return False
            
            if (self.crop_manager.crop_type == "rice" and 
                sensor_data.water_level is not None):
                min_level, max_level = self.crop_manager.parameters["rice"]["water_level"]
                if sensor_data.water_level > max_level:
                    logger.warning(f"Water level too high: {sensor_data.water_level}mm")
                    return False
            
            max_temp = self.crop_manager.parameters[self.crop_manager.crop_type]["temp_max"]
            if sensor_data.temperature > max_temp + 5:
                logger.warning(f"Temperature too high: {sensor_data.temperature}°C")
                return False
            
            return True
            
        except Exception as e:
            logger.error(f"Error in safety checks: {str(e)}")
            return False

    def _monitor_irrigation(self, planned_duration: float, initial_data: SensorData):
        """
        Monitor irrigation process and adjust if needed.

        Parameters
        ----------
        planned_duration : float
            Planned irrigation duration in seconds
        initial_data : SensorData
            Initial sensor readings before irrigation

        Notes
        -----
        Monitors for abnormal conditions and adjusts irrigation accordingly.
        """
        start_time = datetime.now()
        check_interval = 5
        
        while (datetime.now() - start_time).total_seconds() < planned_duration:
            try:
                current_data = self.read_sensors()
                
                if self._detect_abnormal_conditions(current_data, initial_data):
                    logger.warning("Abnormal conditions detected during irrigation")
                    break
                
                if (self.crop_manager.crop_type == "rice" and 
                    current_data.water_level is not None):
                    _, max_level = self.crop_manager.parameters["rice"]["water_level"]
                    if current_data.water_level >= max_level:
                        logger.info("Target water level reached for rice")
                        break
                
                time.sleep(check_interval)
                
            except Exception as e:
                logger.error(f"Error during irrigation monitoring: {str(e)}")
                break

    def _detect_abnormal_conditions(self, current: SensorData, 
                                  initial: SensorData) -> bool:
        """
        Detect abnormal conditions during irrigation.

        Parameters
        ----------
        current : SensorData
            Current sensor readings
        initial : SensorData
            Initial sensor readings before irrigation

        Returns
        -------
        bool
            True if abnormal conditions detected, False otherwise

        Notes
        -----
        Checks for:
        * Sudden flow rate changes (>5 L/min)
        * Excessive moisture increase (>20%)
        * Flooding conditions (>95mm water level)
        """
        try:
            if abs(current.flow_rate - initial.flow_rate) > 5:
                logger.warning("Abnormal flow rate change detected")
                return True
            
            moisture_change = current.soil_moisture - initial.soil_moisture
            if moisture_change > 20:
                logger.warning("Excessive moisture increase detected")
                return True
            
            if current.water_level is not None and current.water_level > 95:
                logger.warning("Flooding condition detected")
                return True
            
            return False
            
        except Exception as e:
            logger.error(f"Error in abnormal condition detection: {str(e)}")
            return True

In [26]:
class IrrigationSystem:
    """
    Main system class for managing multiple irrigation controllers.

    Parameters
    ----------
    None

    Attributes
    ----------
    controllers : Dict[str, IrrigationController]
        Dictionary mapping plot IDs to their respective irrigation controllers
    db_manager : DatabaseManager
        Database manager instance for logging system events

    Notes
    -----
    Central management system responsible for:
    * Controller initialization
    * Plot management
    * System monitoring
    * Water volume calculations
    """
    
    def __init__(self):
        """
        Initialize irrigation system with empty controllers and database manager.

        Parameters
        ----------
        None

        Notes
        -----
        Creates:
        * Empty dictionary for irrigation controllers
        * Database management system for logging
        """
        self.controllers: Dict[str, IrrigationController] = {}
        self.db_manager = DatabaseManager()
        
    def add_plot(self, plot_id: str, crop_type: str, 
                 dimensions: Tuple[float, float], 
                 planting_date: datetime,
                 valve_id: str):
        """
        Add a new plot to the irrigation system.

        Parameters
        ----------
        plot_id : str
            Unique identifier for the plot
        crop_type : str
            Type of crop being grown (e.g., 'rice', 'maize')
        dimensions : Tuple[float, float]
            Plot dimensions as (length, width) in meters
        planting_date : datetime
            Date when the crop was planted
        valve_id : str
            Unique identifier for the irrigation valve

        Notes
        -----
        Creates new IrrigationController instance and logs plot addition
        """
        self.controllers[plot_id] = IrrigationController(
            crop_type, dimensions, planting_date, valve_id)
        logger.info(f"Added new plot {plot_id} with crop type {crop_type}")
        
    def start_system(self):
        """
        Start irrigation system and begin monitoring all plots.

        Parameters
        ----------
        None

        Notes
        -----
        System operation:
        * Creates daemon threads for each controller
        * Starts all controller threads
        * Maintains main thread for health checks
        * Handles graceful shutdown on interrupt

        Raises
        ------
        KeyboardInterrupt
            When system shutdown is requested by user
        """
        logger.info("Starting irrigation system")
        
        # Create threads for each controller
        import threading
        threads = []
        
        for plot_id, controller in self.controllers.items():
            thread = threading.Thread(
                target=controller.control_loop,
                name=f"Controller-{plot_id}"
            )
            thread.daemon = True
            threads.append(thread)
            
        # Start all threads
        for thread in threads:
            thread.start()
            
        try:
            # Keep main thread alive
            while True:
                time.sleep(60)
                self._system_health_check()
                
        except KeyboardInterrupt:
            logger.info("Shutting down irrigation system")
            # Cleanup will happen automatically as threads are daemonic
            
    def _system_health_check(self):
        """
        Perform system-wide health check on all controllers.

        Parameters
        ----------
        None

        Notes
        -----
        Monitors:
        * Valve states for each plot
        * Sensor readings and safety checks
        * System errors and exceptions

        Logging:
        * Active irrigation status
        * Safety check failures
        * System errors
        """
        for plot_id, controller in self.controllers.items():
            try:
                # Check valve states
                if len(controller.valve_controller.valve_states) > 0:
                    logger.info(f"Plot {plot_id} has active irrigation")
                
                # Check sensor readings
                sensor_data = controller.read_sensors()
                if not controller._safety_checks(sensor_data):
                    logger.warning(f"Safety check failed for plot {plot_id}")
                
            except Exception as e:
                logger.error(f"Health check failed for plot {plot_id}: {str(e)}")

    def calculate_water_volume(self, crop_type: str, stage: str, kc: float, duration: int, 
                             eto: float, plot_area: float, eto_source: str) -> float:
        """
        Calculate water volume requirement for a specific growth stage.

        Parameters
        ----------
        crop_type : str
            Type of crop ('rice', 'maize', etc.)
        stage : str
            Growth stage ('initial', 'development', etc.)
        kc : float
            Crop coefficient for the current growth stage
        duration : int
            Duration of the growth stage in days
        eto : float
            Reference evapotranspiration in mm/day
        plot_area : float
            Area of the plot in square meters
        eto_source : str
            Source of the ETo value ('fixed' or 'calculated')

        Returns
        -------
        float
            Required water volume in liters

        Notes
        -----
        Special calculations for rice:
        * Initial stage: +2 L/day for saturation
        * Non-development stages: +6 L/day for percolation
        * Water layer maintenance: +1/3 L/day
        """
        # Base water requirement
        water_req = eto * kc * plot_area * duration
        logger.info(f"Using {eto_source} ETo value: {eto:.2f} mm/day for {crop_type} {stage}")
        
        # Additional requirements for rice
        if crop_type == "rice":
            if stage == "initial":
                water_req += 2 * duration  # Saturation water
            if stage != "development":
                water_req += 6 * duration  # Percolation
                water_req += (1/3) * duration  # Water layer maintenance
        
        return water_req

In [27]:
class TestDataGenerator:
    """
    Generates test data for irrigation system validation.

    A utility class that generates and analyzes water requirements
    for different crops across their growth stages.

    Parameters
    ----------
    system : IrrigationSystem
        The irrigation system instance to analyze
    fixed_eto : float, optional
        Fixed evapotranspiration value to use instead of calculating it
        (default is None)

    Attributes
    ----------
    system : IrrigationSystem
        Reference to the irrigation system being analyzed
    results : dict
        Dictionary storing water requirements for each crop
    fixed_eto : float or None
        Fixed ETo value if provided, None if using calculated values

    Notes
    -----
    Uses standard test conditions:
    * Temperature: 25.0°C
    * Humidity: 60.0%
    * Soil Moisture: 70.0%
    * Water Level: 30.0mm
    * Flow Rate: 15.0 L/min
    """

    def __init__(self, system: IrrigationSystem, fixed_eto: float = None):
        """
        Initialize the test data generator.

        Parameters
        ----------
        system : IrrigationSystem
            The irrigation system instance to analyze
        fixed_eto : float, optional
            Fixed evapotranspiration value to use (default is None)

        Notes
        -----
        Creates empty results dictionary and stores system reference
        """
        self.system = system
        self.results = {}
        self.fixed_eto = fixed_eto

    def generate_stage_requirements(self):
        """
        Calculate water requirements for all crops and growth stages.

        Returns
        -------
        dict
            Dictionary containing water requirements for each crop,
            organized by growth stages and including totals

        Notes
        -----
        Calculation process:
        1. Uses standard test conditions
        2. Calculates or uses fixed ETo value
        3. Computes requirements for each growth stage:
           * Initial
           * Development
           * Mid-season
           * Late-season
        4. Calculates total requirements
        5. Prints formatted table with results

        Table format:
        * Rows: Different crops
        * Columns: Growth stages and total
        * Values: Water requirements in liters
        * Sorted alphabetically by crop name
        """
        logger.info("Generating stage-wise water requirements")

        test_conditions = {
            'temperature': 25.0,
            'humidity': 60.0,
            'soil_moisture': 70.0,
            'water_level': 30.0,
            'flow_rate': 15.0
        }

        # Get ETo value (fixed or calculated)
        eto, eto_source = SimplifiedETo.calc_eto(
            test_conditions['temperature'],
            test_conditions['humidity'],
            fixed_eto=self.fixed_eto
        )

        # Collect results for all crops
        crops_data = {}
        for plot_id, controller in self.system.controllers.items():
            crop_type = controller.crop_manager.crop_type
            stage_reqs = []

            for stage in ['initial', 'development', 'mid_season', 'late_season']:
                kc, duration = controller.crop_manager.stages[crop_type][stage]
                water_req = self.system.calculate_water_volume(
                    crop_type, stage, kc, duration,
                    eto, controller.plot_area, eto_source
                )
                stage_reqs.append(water_req)

            # Add total to stage requirements
            stage_reqs.append(sum(stage_reqs))
            crops_data[crop_type] = stage_reqs
            self.results[crop_type] = dict(zip(
                ['initial', 'development', 'mid_season', 'late_season', 'total'],
                stage_reqs
            ))

        # Print formatted table with units
        print("\nCalculated Water Requirements by Growth Stage:")
        print("-" * 82)
        print(f"{'Crop':<10} {'Initial':>12} {'Development':>12} {'Mid-Season':>12} {'Late-Season':>12} {'Total':>12}")
        print(f"{'':10} {'(L)':>12} {'(L)':>12} {'(L)':>12} {'(L)':>12} {'(L)':>12}")
        print("-" * 82)

        # Sort crops alphabetically
        for crop in sorted(crops_data.keys()):
            values = [f"{val:>12.2f}" for val in crops_data[crop]]
            print(f"{crop:<10} {' '.join(values)}")

        print("-" * 82)
        return self.results

In [28]:
class IntercroppingAnalyzer:
    """
    Analyzes water requirements for intercropping combinations.

    A specialized analyzer that calculates water savings potential
    when different crops are grown together using intercropping methods.

    Parameters
    ----------
    test_generator : TestDataGenerator
        Instance of TestDataGenerator to provide base water requirements

    Attributes
    ----------
    test_generator : TestDataGenerator
        Reference to the test data generator instance
    interaction_factors : dict
        Dictionary of crop combination tuples and their water reduction factors
        Format: (crop1, crop2): reduction_factor
        Example: ('maize', 'beans'): 0.85 means 15% water reduction

    Notes
    -----
    Supported crop combinations and water reductions:
    * Maize + Beans: 15% reduction
    * Maize + Onions: 5% reduction
    * Beans + Onions: 10% reduction
    
    All combinations are bi-directional (order doesn't matter)
    """

    def __init__(self, test_generator: TestDataGenerator):
        """
        Initialize the intercropping analyzer.

        Parameters
        ----------
        test_generator : TestDataGenerator
            Instance of TestDataGenerator to use for calculations

        Notes
        -----
        Sets up interaction factors dictionary for common crop combinations
        Reduction factors represent water savings from beneficial interactions
        """
        self.test_generator = test_generator
        # Define interaction factors - water reduction due to beneficial interactions
        self.interaction_factors = {
            ('maize', 'beans'): 0.85,  # 15% water reduction
            ('beans', 'maize'): 0.85,
            ('onions', 'maize'): 0.95,  # 5% water reduction
            ('maize', 'onions'): 0.95,
            ('beans', 'onions'): 0.90,  # 10% water reduction
            ('onions', 'beans'): 0.90
        }

    def analyze_combinations(self):
        """
        Analyze water requirements for different intercropping combinations.

        Returns
        -------
        None
            Prints analysis results to console

        Notes
        -----
        Analysis process:
        1. Generates base water requirements for individual crops
        2. Analyzes common intercropping combinations:
           * Maize + Beans
           * Maize + Onions
           * Beans + Onions
        3. For each combination:
           * Calculates individual water requirements
           * Applies interaction factors
           * Computes water savings
           * Determines saving percentages
        4. Provides total potential water savings

        Output format:
        * Table with columns:
          - Crop Combination
          - Individual Sum (L)
          - Combined Requirement (L)
          - Water Saving (L)
          - Saving Percentage (%)
        * Summary of total potential savings
        * Explanatory notes

        Raises
        ------
        Exception
            Logs error and prints message if analysis fails
        """
        logger.info("Analyzing intercropping combinations")

        try:
            # Get base requirements
            print("\nWater Requirements Summary:")
            print("-" * 90)
            base_reqs = self.test_generator.generate_stage_requirements()

            # Print intercropping analysis header
            print("\nIntercropping Analysis Results:")
            print("-" * 100)
            print(f"{'Crop Combination':<20} {'Individual Sum':>15} {'Combined Req':>15} {'Water Saving':>15} {'Saving %':>15}")
            print(f"{'':20} {'(L)':>15} {'(L)':>15} {'(L)':>15} {'(%)':>15}")
            print("-" * 100)

            # Analyze common intercropping combinations
            combinations = [
                ('maize', 'beans'),
                ('maize', 'onions'),
                ('beans', 'onions')
            ]

            total_savings = 0
            for crop1, crop2 in combinations:
                # Calculate individual total requirements
                total1 = base_reqs[crop1]['total']
                total2 = base_reqs[crop2]['total']
                individual_sum = total1 + total2

                if individual_sum > 0:
                    # Calculate combined requirement with interaction factor
                    interaction_factor = self.interaction_factors.get((crop1, crop2), 1.0)
                    combined_req = individual_sum * interaction_factor

                    # Calculate savings
                    water_saving = individual_sum - combined_req
                    saving_percent = (water_saving / individual_sum) * 100
                    total_savings += water_saving

                    # Print results with crop names properly formatted
                    combo_name = f"{crop1}+{crop2}"
                    print(f"{combo_name:<20} {individual_sum:>15.2f} {combined_req:>15.2f} "
                          f"{water_saving:>15.2f} {saving_percent:>14.1f}%")

            print("-" * 100)
            print(f"{'Total potential savings':>50}: {total_savings:>15.2f} L")
            print("-" * 100)

            # Print explanatory notes
            print("\nNotes:")
            print("- Individual Sum: Total water requirement if crops are grown separately")
            print("- Combined Req: Water requirement when crops are intercropped")
            print("- Water Saving: Reduction in water use due to intercropping")
            print("- Saving %: Percentage of water saved through intercropping")

        except Exception as e:
            logger.error(f"Error in intercropping analysis: {str(e)}")
            print("Error occurred during analysis. Check logs for details.")

In [29]:
class NotebookIrrigationSystem:
    """
    Jupyter notebook interface for the irrigation system.

    Provides an interactive interface for analyzing and monitoring
    irrigation systems using IPython widgets in a Jupyter environment.

    Attributes
    ----------
    system : IrrigationSystem
        The main irrigation system instance
    option_dropdown : widgets.Dropdown
        Dropdown menu for selecting analysis type
    use_fixed_eto : widgets.Checkbox
        Checkbox for toggling fixed ETo value usage
    eto_value : widgets.FloatText
        Input field for fixed ETo value
    run_button : widgets.Button
        Button to execute selected analysis
    output : widgets.Output
        Widget for displaying analysis results
    widget_box : widgets.VBox
        Container for all interface widgets

    Notes
    -----
    Supported analysis options:
    * Water Requirement Analysis
    * Intercropping Analysis
    * Real-time Monitoring

    Default plot configuration:
    * Plot 1: Rice (0.6m x 0.6m)
    * Plot 2: Maize (0.6m x 0.6m)
    * Plot 3: Beans (0.6m x 0.6m)
    * Plot 4: Onions (0.6m x 0.6m)
    """

    def __init__(self):
        """
        Initialize the notebook interface.

        Notes
        -----
        Performs initialization steps:
        * Creates irrigation system instance
        * Sets up default plots
        * Creates interactive widgets
        """
        self.system = IrrigationSystem()
        self.setup_plots()
        self.create_widgets()

    def setup_plots(self):
        """
        Initialize default plots in the system.

        Notes
        -----
        Creates four default plots:
        * Plot 1: Rice, planted March 1, 2024
        * Plot 2: Maize, planted March 15, 2024
        * Plot 3: Beans, planted March 10, 2024
        * Plot 4: Onions, planted March 5, 2024

        All plots are 0.6m x 0.6m with dedicated valves
        """
        plots = [
            ("plot1", "rice", (0.6, 0.6), datetime(2024, 3, 1), "valve1"),
            ("plot2", "maize", (0.6, 0.6), datetime(2024, 3, 15), "valve2"),
            ("plot3", "beans", (0.6, 0.6), datetime(2024, 3, 10), "valve3"),
            ("plot4", "onions", (0.6, 0.6), datetime(2024, 3, 5), "valve4")
        ]
        for plot_id, crop_type, dimensions, planting_date, valve_id in plots:
            self.system.add_plot(plot_id, crop_type, dimensions, planting_date, valve_id)

    def create_widgets(self):
        """
        Create interactive widgets for the notebook interface.

        Notes
        -----
        Creates and configures:
        * Analysis type dropdown
        * ETo configuration widgets
        * Run button
        * Output display area

        Widget layout:
        * Vertical arrangement of components
        * Horizontal arrangement of ETo settings
        * Configures callbacks for interactivity
        """
        # Main option selection
        self.option_dropdown = widgets.Dropdown(
            options=[
                ('Water Requirement Analysis', 1),
                ('Intercropping Analysis', 2),
                ('Real-time Monitoring', 3)
            ],
            description='Analysis:',
            style={'description_width': 'initial'}
        )

        # ETo input widgets
        self.use_fixed_eto = widgets.Checkbox(
            value=False,
            description='Use fixed ETo',
            style={'description_width': 'initial'}
        )
        self.eto_value = widgets.FloatText(
            value=5.0,
            description='ETo (mm/day):',
            disabled=True,
            style={'description_width': 'initial'}
        )

        # Run button
        self.run_button = widgets.Button(
            description='Run Analysis',
            button_style='success'
        )

        # Output widget for results
        self.output = widgets.Output()

        # Wire up the callbacks
        self.use_fixed_eto.observe(self._toggle_eto_input, names='value')
        self.run_button.on_click(self._run_analysis)

        # Layout the widgets
        self.widget_box = widgets.VBox([
            self.option_dropdown,
            widgets.HBox([self.use_fixed_eto, self.eto_value]),
            self.run_button,
            self.output
        ])

    def _toggle_eto_input(self, change):
        """
        Enable/disable ETo input based on checkbox state.

        Parameters
        ----------
        change : traitlets.utils.bunch.Bunch
            Change event object containing the new checkbox state

        Notes
        -----
        Enables ETo input field when checkbox is checked,
        disables it when unchecked
        """
        self.eto_value.disabled = not change.new

    def _run_analysis(self, _):
        """
        Run the selected analysis type with current settings.

        Parameters
        ----------
        _ : widgets.Button
            Button click event object (unused)

        Notes
        -----
        Analysis types:
        * Option 1: Water requirement analysis
        * Option 2: Intercropping analysis
        * Option 3: Real-time monitoring

        Handles errors by:
        * Displaying error message
        * Logging error details
        """
        with self.output:
            clear_output()
            try:
                # Get analysis parameters
                fixed_eto = self.eto_value.value if self.use_fixed_eto.value else None
                test_generator = TestDataGenerator(self.system, fixed_eto)

                if self.option_dropdown.value == 1:
                    # Water requirement analysis
                    test_generator.generate_stage_requirements()
                elif self.option_dropdown.value == 2:
                    # Intercropping analysis
                    intercrop_analyzer = IntercroppingAnalyzer(test_generator)
                    intercrop_analyzer.analyze_combinations()
                elif self.option_dropdown.value == 3:
                    print("Starting real-time monitoring...")
                    print("(Note: In a notebook environment, this will run until interrupted)")
                    self.system.start_system()

            except Exception as e:
                print(f"Error during analysis: {str(e)}")
                logger.error(f"Analysis error: {str(e)}")

    def display(self):
        """
        Display the interactive interface in the notebook.

        Notes
        -----
        Shows the complete widget interface including:
        * Analysis selection dropdown
        * ETo configuration options
        * Run button
        * Results output area
        """
        display(self.widget_box)

In [30]:
"""
Initialize and display the irrigation system's interactive notebook interface.

Notes
-----
Provides:
* Analysis type selection
* ETo configuration
* Results display
"""

# Create and display the interactive system
irrigation_system = NotebookIrrigationSystem()
irrigation_system.display()

2024-10-29 00:44:54,601 - IrrigationSystem - INFO - Added new plot plot1 with crop type rice
2024-10-29 00:44:54,614 - IrrigationSystem - INFO - Added new plot plot2 with crop type maize
2024-10-29 00:44:54,616 - IrrigationSystem - INFO - Added new plot plot3 with crop type beans
2024-10-29 00:44:54,618 - IrrigationSystem - INFO - Added new plot plot4 with crop type onions
